# Twitter recommender system 

This is a simple recommender system, to get similar tweets for Samsung and iPhone based on top tfidf scores of the tweets:

    Input : Giving twitter id and range of top tweet number.
    
    output:    
        1.Recommanding similar tweets as same sentiment of the input tweet.        
        2.Recommanding similar tweets as opposit sentiment of the input tweet.        
        3.Recommanding similar tweets as input tweet with Samsung brand.        
        4.Recommanding similar tweets as input tweet with iPhone brand tweet.


In [9]:
import pandas as pd
import numpy as np
import re
from IPython.display import display, HTML, Markdown
from datetime import datetime, timedelta
from sklearn.feature_extraction.text import TfidfVectorizer
import sys
#from sklearn.metrics.pairwise import linear_kernel
import sys
import warnings

if not sys.warnoptions:
    warnings.simplefilter("ignore")


Reading 2 brands combined preprocessed data file

In [10]:
tweets_main = pd.read_csv("F:/web mining/webmining project/crawler/twitter_Web_Mining/Data/Combined_final_14may.csv")


In [11]:
#user name extraction
tweets_main.insert(1,'user_name','name')
for i in range(0, len(tweets_main)):
    link = tweets_main.permalink[i]
    df_name = re.search(r'https://twitter.com/([^/?]+)', link).group(1)
    tweets_main.at[i, 'user_name'] = df_name

In [12]:
#create user Id
tweets_main.insert(1,'user_name_id',0)
for i in range(0, len(tweets_main)):
     tweets_main.at[i, 'user_name_id'] = i+1

In [13]:
#date time type conversion
tweets_main['date'] = pd.to_datetime(tweets_main['date'])
tweets_main.insert(1,'only_date','0000-00-00')

In [14]:
#date extraction
for i in range(0, len(tweets_main)):
    date_time = tweets_main.date[i]
    tweets_main.at[i, 'only_date'] = date_time.date()

In [15]:
tweet_data = tweets_main.loc[tweets_main['id'] == 925473978625548288 ]
date = tweet_data['only_date'] - timedelta(days=1)
d = date.values
index = tweets_main[tweets_main['only_date'].isin(d)].index.tolist()
tweets = tweets_main.loc[index]

In [24]:
def getrecommendation_basedOn_date(id,top):
    similarPosTweets = []
    similarNegTweets = []
    similarSamTweets = []
    similarIPhTweets = []
    c=0
    for i in range(0, len(tweets_main)):
        if tweets_main.id[i] == id:
            #SAMSUNG
            tmp_brand = tweets_main.brand[i]
            if tweets_main.brand[i] == 'Samsung':
                if tweets_main.polarity[i] == 'positive':
                    print("Given tweet ID is a Positive tweet for the brand Samsung.")
                    print("Below are previous days top similar tweets.")
                elif tweets_main.polarity[i] == 'negative':
                    print("Given tweet ID is a Negative tweet for the brand Samsung.")
                    print("Below are previous days top similar tweets.")
                else:
                    sys.exit("Given tweet ID is a Neutral tweet. Kindly give only positive or negative tweets ID.")
            #IPHONE
            if tweets_main.brand[i] == 'iphone':
                tmp_brand = tweets_main.brand[i]
                if tweets_main.polarity[i] == 'positive':
                    print("It is a Positive tweet for the brand IPhone.")
                    print("Below are previous days top similar tweets.")
                elif tweets_main.polarity[i] == 'negative':
                    print("It is a Negative tweet for the brand IPhone.")
                    print("Below are previous days top similar tweets.")
                else:
                    sys.exit("It is a Neutral tweet. Kindly give only positive or negetive tweets ID.")
                    
                    
    tweet_data = tweets_main.loc[tweets_main['id'] == id ]
    date = tweet_data['only_date'] - timedelta(days=1)
    d = date.values
    index = tweets_main[tweets_main['only_date'].isin(d)].index.tolist()
    tweets = tweets_main.loc[index]    
    #only positive and negative tweets are considered. 
    tweets_PosNeg = tweets[(tweets.polarityNum != 0.0)]
    tfidf = TfidfVectorizer()

    #Replace NaN with an empty string
    tweets_PosNeg['text_preprocessed'] = tweets_PosNeg['text_preprocessed'].fillna('')

    #Construct the required TF-IDF matrix by fitting and transforming the data
    tfidf_matrix_PosNeg = tfidf.fit_transform(tweets_PosNeg['text_preprocessed'])

    # Compute the cosine similarity matrix
    cosine_sim_PosNeg = linear_kernel(tfidf_matrix_PosNeg, tfidf_matrix_PosNeg)
    
    indices_PosNeg = pd.Series(tweets_PosNeg.index, index=tweets_PosNeg['id']).drop_duplicates()
    
    for k in range(0,len(indices_PosNeg)):       
        if (indices_PosNeg.index[k] == id):
            break;
        c+= 1
    c-= 1
    sim_scores = list(enumerate(cosine_sim_PosNeg[c]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    #print(sim_scores)
    for j in sim_scores:
        ind = (j[0])
        if (tweets_PosNeg['polarity'].iloc[ind] == 'positive' and tweets_PosNeg['brand'].iloc[ind] == tmp_brand):
            similarPosTweets.append([tweets_PosNeg['id'].iloc[ind] ,tweets_PosNeg['only_date'].iloc[ind] ,tweets_PosNeg['text_original'].iloc[ind], tweets_PosNeg['polarity'].iloc[ind],tweets_PosNeg['brand'].iloc[ind]])
        elif (tweets_PosNeg['polarity'].iloc[ind] == 'negative' and tweets_PosNeg['brand'].iloc[ind] == tmp_brand):
            similarNegTweets.append([tweets_PosNeg['id'].iloc[ind] ,tweets_PosNeg['only_date'].iloc[ind] , tweets_PosNeg['text_original'].iloc[ind], tweets_PosNeg['polarity'].iloc[ind],tweets_PosNeg['brand'].iloc[ind]])
        if (tweets_PosNeg['brand'].iloc[ind] == 'Samsung'):
            similarSamTweets.append([tweets_PosNeg['id'].iloc[ind] ,tweets_PosNeg['only_date'].iloc[ind] , tweets_PosNeg['text_original'].iloc[ind], tweets_PosNeg['polarity'].iloc[ind],tweets_PosNeg['brand'].iloc[ind]])
        elif (tweets_PosNeg['brand'].iloc[ind] == 'iphone'):
            similarIPhTweets.append([tweets_PosNeg['id'].iloc[ind] ,tweets_PosNeg['only_date'].iloc[ind] , tweets_PosNeg['text_original'].iloc[ind], tweets_PosNeg['polarity'].iloc[ind],tweets_PosNeg['brand'].iloc[ind]])
    similarPosTweets = similarPosTweets[1:(top+1)]
    similarNegTweets = similarNegTweets[1:(top+1)]
    similarSamTweets = similarSamTweets[1:(top+1)]
    similarIPhTweets = similarIPhTweets[1:(top+1)]
    
    df_similarPosTweets = pd.DataFrame(np.array(similarPosTweets).reshape(-1,5), columns = list("abcde"))
    df_similarNegTweets = pd.DataFrame(np.array(similarNegTweets).reshape(-1,5), columns = list("abcde"))
    df_similarSamTweets = pd.DataFrame(np.array(similarSamTweets).reshape(-1,5), columns = list("abcde"))
    df_similarIPhTweets = pd.DataFrame(np.array(similarIPhTweets).reshape(-1,5), columns = list("abcde"))


    df_similarPosTweets = df_similarPosTweets.rename(index=str, columns={"a": "Id", "b": "Date", "c" : "Tweet","d" : "Polarity", "e" : "Brand"})
    df_similarNegTweets = df_similarNegTweets.rename(index=str, columns={"a": "Id", "b": "Date", "c" : "Tweet","d" : "Polarity", "e" : "Brand"})
    df_similarSamTweets = df_similarSamTweets.rename(index=str, columns={"a": "Id", "b": "Date", "c" : "Tweet","d" : "Polarity", "e" : "Brand"})
    df_similarIPhTweets = df_similarIPhTweets.rename(index=str, columns={"a": "Id", "b": "Date", "c" : "Tweet","d" : "Polarity", "e" : "Brand"})
    print('\033[91m' + '\033[1m' + '**************************************Similar positive tweets from the same brand**************************************')
    display(df_similarPosTweets)
    print('\033[91m' + '\033[1m' + '**************************************Similar negative tweets from the same brand**************************************')
    display(df_similarNegTweets)
    print('\033[91m' + '\033[1m' + '**************************************Similar Samsung positive and negative tweets**************************************')
    display(df_similarSamTweets)
    print('\033[91m' + '\033[1m' + '**************************************Similar iPhone positive and negative tweets**************************************')
    display(df_similarIPhTweets) 

In [25]:
getrecommendation_basedOn_date(925887767766274048, 10)

Given tweet ID is a Negative tweet for the brand Samsung.
Below are previous days top similar tweets.
**************************************Similar positive tweets from the same brand**************************************


,Id,Date,Tweet,Polarity,Brand
0,925828335640379392,2017-11-01,Looks a little thick. It'll be interesting to ...,positive,Samsung
1,925543381710749696,2017-11-01,"Hi, Everyone. New iPhone X/8/8P and Samsung No...",positive,Samsung
2,925566913740238848,2017-11-01,BEST TOP EBAY AND AMAZON DEALS!! CLICK http://...,positive,Samsung
3,925832623519404032,2017-11-01,"My current one, Samsung Galaxy Note8. Why? Bec...",positive,Samsung
4,925849305461686273,2017-11-01,Can someone explain why apple has so great pho...,positive,Samsung
5,925837596806262784,2017-11-01,The Samsung Galaxy S8 is quite possibly the mo...,positive,Samsung
6,925574072175661056,2017-11-01,I just got my new phone (Samsung Galaxy S7) an...,positive,Samsung
7,925846901576093696,2017-11-01,I don’t WANT to pay it. But the good Samsung o...,positive,Samsung
8,925586245530587137,2017-11-01,Kinda want to go back to my samsung and change...,positive,Samsung
9,925685850817560576,2017-11-01,I love the phone but you can't trust anybody's...,positive,Samsung


**************************************Similar negative tweets from the same brand**************************************


,Id,Date,Tweet,Polarity,Brand
0,925742372209807360,2017-11-01,I would like you to explain to us why Samsung'...,negative,Samsung
1,925814488913317889,2017-11-01,"its made so people keep buying the phones, not...",negative,Samsung
2,925598614990311425,2017-11-01,Dear Samsung. I officially hate my phone and w...,negative,Samsung
3,925570365476159488,2017-11-01,this phone is literal garbage like samsung nee...,negative,Samsung
4,925829526352887808,2017-11-01,My samsung phone does not have the new # dinos...,negative,Samsung
5,925751780830351360,2017-11-01,@ AppleSupport what happened to your devices! ...,negative,Samsung
6,925608126883291136,2017-11-01,the pixel 2 has a camera that no other smart p...,negative,Samsung
7,925538002675187712,2017-11-01,So my process is as I pull the phone out of my...,negative,Samsung
8,925705879269363712,2017-11-01,"Baby I can’t even lie to you, my phones been b...",negative,Samsung
9,925537548356665344,2017-11-01,Unlocking is the authentication. If my phone i...,negative,Samsung


**************************************Similar Samsung positive and negative tweets**************************************


,Id,Date,Tweet,Polarity,Brand
0,925742372209807360,2017-11-01,I would like you to explain to us why Samsung'...,negative,Samsung
1,925814488913317889,2017-11-01,"its made so people keep buying the phones, not...",negative,Samsung
2,925740964530049025,2017-11-01,ex @ Apple # iPhone user. @ Android won me bac...,positive,Samsung
3,925828335640379392,2017-11-01,Looks a little thick. It'll be interesting to ...,positive,Samsung
4,925543381710749696,2017-11-01,"Hi, Everyone. New iPhone X/8/8P and Samsung No...",positive,Samsung
5,925566913740238848,2017-11-01,BEST TOP EBAY AND AMAZON DEALS!! CLICK http://...,positive,Samsung
6,925598614990311425,2017-11-01,Dear Samsung. I officially hate my phone and w...,negative,Samsung
7,925570365476159488,2017-11-01,this phone is literal garbage like samsung nee...,negative,Samsung
8,925829526352887808,2017-11-01,My samsung phone does not have the new # dinos...,negative,Samsung
9,925832623519404032,2017-11-01,"My current one, Samsung Galaxy Note8. Why? Bec...",positive,Samsung


**************************************Similar iPhone positive and negative tweets**************************************


,Id,Date,Tweet,Polarity,Brand
0,925787061591912448,2017-11-01,I love the iPhone update personally. I love wh...,positive,iphone
1,925787061591912448,2017-11-01,I love the iPhone update personally. I love wh...,positive,iphone
2,925807558798999553,2017-11-01,i want to watch shane’s video but my phone is ...,negative,iphone
3,925717070456561664,2017-11-01,Yes im starving BUT how can i eat knowing that...,negative,iphone
4,925554436298113024,2017-11-01,Yo @ Apple how come when the IPhone X coming o...,positive,iphone
5,925813021401862145,2017-11-01,every 2 or so years you gotta buy a new iphone...,positive,iphone
6,925838767063207936,2017-11-01,the only thing i hate abt having an iPhone 7 +...,negative,iphone
7,925745646065324032,2017-11-01,Oi APPLE...with the new iPhone you can’t charg...,positive,iphone
8,925649811134648320,2017-11-01,When Millie gets a new phone for her birthday ...,positive,iphone
9,925812968033419264,2017-11-01,"I really hate that Apple, took the headphone j...",negative,iphone
